<a href="https://colab.research.google.com/github/smk51/Aqueduct3.0_RepRisk/blob/master/Data_Processing/historic_eia_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Water Peace and Security Data Prep:**
# **National Data**
# **Annual Update**




```
Project: Water Peace and Security Initiative Pillar 1A
Date: July 15, 2020
Author: Samantha Kuzma (samantha.kuzma@wri.org)
```
---

In [ ]:
# serbia 1994-2005 copy & save as montenegro (MNE)

# Background Info (import modules, connect to bucket, load functions)

In [ ]:
# LIST OF FUNCTIONS
# - - - READ IN DATA
# Read files on GCS into Pandas Dataframe (GCS = Google Cloud Storage)
def gcs_to_pandas(file_type):
  # Find the index of the desired file
  file_ind = [files.index(i) for i in files if file_type in str(i)]
  # Select the file
  selected_file = files[file_ind[0]]
  # Download data from file
  file_contents = selected_file.download_as_string()
  # Return contents of file (as string)
  return file_contents

def read_data(filename, head, col): 
  selected_content = gcs_to_pandas(filename)
  df = pd.read_csv(io.BytesIO(selected_content), header=head, index_col = col, encoding='latin-1')
  return df

def save_file(outDF, outname, main_folder, sub_folder):
  filename_reserved = outname
  outDF.to_csv(filename_reserved)
  !gsutil cp $filename_reserved gs://{bucket_name}/{main_folder}/{sub_folder}/{vnumber}/
  !gsutil ls gs://{bucket_name}/{main_folder}/{sub_folder}/{vnumber}/
  print('save complete!')

def make_dataframe(year_range, country_list):
  # Create a dataframe to hold all places and years
  # World bank data will be joined onto this dataframe
  # Find number of years and places
  numYEARS = len(range(year_range[0], year_range[-1]+1)) # find number of years
  numPLACES = len(country_list) #find number of places
  # create series of years for each country
  all_years  = [[i]*numPLACES for i in range(year_range[0], year_range[-1]+1)]
  y_flat = [str(y) for x in all_years for y in x]

  # Create dataframe to hold the data
  df = pd.DataFrame(index=range(0, numYEARS * numPLACES))
  df['Country'] = country_list * numYEARS
  df['Year'] = y_flat
  df.set_index(['Country', 'Year'], inplace=True)
  return df
 

In [ ]:
# READ IN LIBRARIES
# Python Libraries
from google.colab import auth
import uuid
import pandas as pd
import numpy as np
import io
import datetime

# Use the GCS Python API to read all the files in our bucket.
!pip install -q EIA_python
import eia

# CONNECT TO CLOUD
# Authenticate Google Cloud Storage
auth.authenticate_user()

# LINK TO GOOGLE BUCKET
# Project ID for WPSI bucket
project_id = 'wpsi-208318'

!gcloud config set project {project_id}

# WPSI bucket name
bucket_name = 'wps_pillar1a'

# Use the GCS Python API to read all the files in our bucket.
!pip install -q google-cloud-storage 
from google.cloud import storage

gcs_client = storage.Client(project=project_id)
bucket = gcs_client.bucket(bucket_name)

# Create a list of all files in the bucket
files = list(bucket.list_blobs())

Updated property [core/project].


In [ ]:
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    df = pd.DataFrame(series_search)
    return df

def main(series_ID, gid_, eid_):
    """
    Run main script
    """
    #Create EIA API using your specific API key
    api_key = "1dac9ef1f44288454f71865ae997ae10" #Sam Kuzma's key
    api = eia.API(api_key)
    #Declare desired series ID
    # series_ID='EMISS.CO2-TOTV-TT-NG-TX.A'
    df = retrieve_time_series(api, series_ID)
    df.columns = [eid_]
    df['Country'] = gid_
    df.reset_index(inplace=True)
    df['Year'] = df['index'].apply(lambda x: str(int(x)))
    df.drop(['index'], axis=1, inplace=True)
    df.set_index(['Country', 'Year'],inplace=True)
    #Print the returned dataframe df
    return df

In [ ]:
# Read in lookuptable with all API Keys
df_lk = read_data("eia_key_lookup.csv", 0, None)

In [ ]:
# create list of countrys
countries = list(set(df_lk.GID_0.tolist()))
cols = list(set(df_lk.Indicator_id.tolist()))
df_main = make_dataframe([2004, 2017], countries)
# Add empty columns to dataframe
df_main = df_main.reindex(df_main.columns.tolist() + cols, axis=1)

In [ ]:
for i in range(0, len(df_lk)):
  print(str(i))
  skey = df_lk.loc[i, 'Key'] #API Key
  gid = df_lk.loc[i, 'GID_0'] #Country Code
  eid = df_lk.loc[i, 'Indicator_id'] #Indicator ID
  try:
    # Pull data using API
    df_i = main(skey, gid, eid)
    # Update data in main table
    df_main.update(df_i)
  except:
    print(str(i), 'FAILED')

vnumber = 'v6'
save_file(df_main, "historic_eia.csv", "data_working", 'update_model_data')

In [ ]:
vnumber = 'v6'
save_file(df_main, "historic_eia.csv", "data_working", 'update_model_data')

Copying file://historic_eia.csv [Content-Type=text/csv]...
/ [1 files][830.9 KiB/830.9 KiB]                                                
Operation completed over 1 objects/830.9 KiB.                                    
gs://wps_pillar1a/data_working/update_model_data/v6/historic_eia.csv
gs://wps_pillar1a/data_working/update_model_data/v6/spam_agri_prd_GID_0.csv
gs://wps_pillar1a/data_working/update_model_data/v6/spam_agri_prd_GID_1.csv
gs://wps_pillar1a/data_working/update_model_data/v6/spam_agri_prd_GID_2.csv
gs://wps_pillar1a/data_working/update_model_data/v6/spam_agri_val_GID_0.csv
gs://wps_pillar1a/data_working/update_model_data/v6/spam_agri_val_GID_1.csv
gs://wps_pillar1a/data_working/update_model_data/v6/spam_agri_val_GID_2.csv
gs://wps_pillar1a/data_working/update_model_data/v6/wits_data.csv
gs://wps_pillar1a/data_working/update_model_data/v6/world_bank_data.csv
save complete!


In [ ]:
df_main

,,elec_capc_hydr,elec_genr_totl,eng_cons_coal,elec_capc_rnew,elec_genr_nucl,eng_prod_totl,eng_prod_coal,elec_genr_wind,elec_capc_foss,elec_genr_solr,eng_cons_natg,eng_cons_pcap,eng_cons_pgdp,elec_capc_nucl,elec_genr_tide,eng_cons_petr,elec_genr_rnew,elec_expt_totl,elec_genr_foss,eng_prod_natg,eng_prod_petr,elec_capc_totl,elec_impt_totl,elec_genr_biom,elect_dist_loss,eng_cons_othr,eng_cons_totl,elec_capc_geom,elec_capc_tide,elec_genr_geom,elec_capc_biom,elec_capc_solr,elec_genr_hydr,eng_prod_othr,elec_capc_wind
Country,Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
HRV,2004,1.79,12.696,0.0291237,1.799,0,0.178477,0,0.002,1.915,0,0.101681,94.2588,4.63253,0,0,0.195337,6.894,2.296,5.842,0.0588433,0.050583,4.007,5.339,0.004,2.224,0.0800137,0.406155,0,0,0,0.003,0,6.888,0.0690503,0.006
COL,2004,8.917,48.778,0.0930186,9.007,0,3.14205,1.35978,0.052,4.458,0.007,0.208657,29.5426,3.03633,0,0,0.567072,40.202,1.682,8.576,0.208657,1.17095,13.465,0.048,0.467,9.704,0.374286,1.24303,0,0,0,0.08,0,39.676,0.402663,0.01
CYP,2004,0,3.948,0.000588654,0,0,0,0,0,0.994,0,0,145.151,4.06572,0,0,0.105816,0,0,3.948,0,0,0.994,0,0,0.213,-3.5e-41,0.106405,0,0,0,0,0,0,0,0
BFA,2004,0.032,0.472,0,0.0331,0,0.0010016,0,0,0.181,0,0,1.29385,0.897682,0,0,0.0155305,0.1,0,0.372,0,0,0.2141,0.096,0,0.03304,0.00132915,0.0168596,0,0,0,0,0.0011,0.1,0.0010016,0
SGP,2004,0,35.559,1.02558e-05,0.12,0,0.00959533,0,0,9.509,0,0.244403,496.299,7.7897,0,0,1.81365,0.958,0,34.601,0,0,9.629,0,0.958,2.167,0.00984758,2.06791,0,0,0,0.12,0,0,0.00959533,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IND,2017,40.579,1438.17,16.6235,100.828,34.848,17.6759,12.0645,51.06,276.293,26.035,2.08674,22.7349,3.26615,6.255,0,8.96023,247.536,7.203,1155.78,1.16611,1.7438,388.161,5.611,45.431,259.827,2.80589,30.4763,0,0,0,9.5,17.87,125.01,2.70151,32.879
PRI,2017,0.098,16.691,0.0182583,0.369,0,0.0033996,0,0.149,5.934,0.159,0.0486152,83.579,3.79507,0,0,0.194144,0.369,0,16.322,0,0,6.303,0,0.011,2.5528,0.0033996,0.264417,0,0,0,0.005,0.165,0.05,0.0033996,0.101
MNG,2017,0.028,5.69,0.165147,0.177,0,1.12576,1.07824,0.309,1.273,0.029,0,73.6547,5.91796,0,0,0.05526,0.396,0.026,5.294,0,0.043873,1.45,1.574,0,0.811,0.00893886,0.229345,0,0,0,0,0.048,0.058,0.00364835,0.101
